## Ingestion

In [1]:
import pandas as pd
import json
from tqdm.auto import tqdm
from sentence_transformers import SentenceTransformer
from elasticsearch import Elasticsearch

In [2]:
df = pd.read_csv('../data/fitness_exercises_500.csv')

In [3]:
df.head()

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instruction
0,0,Push-Up Hold,mobility,dip belt,lower,stretch,"glutes, quads, hamstrings",Setup: Prepare equipment: dip belt. Ensure sta...
1,1,Explosive Lateral Raise Pulse,cardio,barbells,full body,hold,"back, chest, legs",Setup: Set barbell with collars secured; grip ...
2,2,Rotational Jumping Jack Iso,warm-up,barbells,upper,push,"biceps, forearms, chest",Setup: Set barbell with collars secured; grip ...
3,3,Wide-Grip Running,strength,kettlebells,upper,pull,"deltoids, triceps, forearms",Setup: Place kettlebell close to midline; hing...
4,4,Decline Dips,cardio,barbells,core,stretch,"abs, lower back, obliques",Setup: Set barbell with collars secured; grip ...


In [4]:
# import minsearch
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [5]:
df.columns

Index(['id', 'exercise_name', 'type_of_activity', 'type_of_equipment',
       'body_part', 'type', 'muscle_groups_activated', 'instruction'],
      dtype='object')

In [6]:
documents = df.to_dict(orient='records')

In [7]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Setup: Prepare equipment: dip belt. Ensure stable setup and safe load. Brace core, keep neutral spine, drive through whole foot. Primary focus: glutes, quads, hamstrings. Movement: Gently enter the stretch until a mild pull is felt; hold without bouncing and keep breathing. Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Slow nasal breaths; exhale to deepen slightly, no pain. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid bouncing; stay within mild discomfort. Do not bounce or hyperextend joints. Safety: Never force end range; joints should feel stable.'}

In [8]:
for doc in tqdm(documents):
    exercise_name = doc['exercise_name']
    instruction  = doc['instruction']
    ei = exercise_name + " " + instruction

    doc['exercise_name_vector'] = model.encode(exercise_name)
    doc['instruction_vector'] = model.encode(instruction)
    doc['ei_vector'] = model.encode(ei)

  0%|          | 0/500 [00:00<?, ?it/s]

In [9]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Setup: Prepare equipment: dip belt. Ensure stable setup and safe load. Brace core, keep neutral spine, drive through whole foot. Primary focus: glutes, quads, hamstrings. Movement: Gently enter the stretch until a mild pull is felt; hold without bouncing and keep breathing. Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Slow nasal breaths; exhale to deepen slightly, no pain. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid bouncing; stay within mild discomfort. Do not bounce or hyperextend joints. Safety: Never force end range; joints should feel stable.',
 'exercise_name_vector': array([-8.45749974e-02, -1.95661839e-02, -1.47137083e-02,  4.42054011e-02,
        -7.81097487

In [18]:
query = 'give me leg exercises for hamstrings'

## Implementing RAG Flow

In [19]:
from google import genai

In [20]:
client = genai.Client()

In [21]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        boost_dict=boost,
        filter_dict={},
        num_results=10
    )

    return results

In [22]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Setup: Prepare equipment: dip belt. Ensure stable setup and safe load. Brace core, keep neutral spine, drive through whole foot. Primary focus: glutes, quads, hamstrings. Movement: Gently enter the stretch until a mild pull is felt; hold without bouncing and keep breathing. Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Slow nasal breaths; exhale to deepen slightly, no pain. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid bouncing; stay within mild discomfort. Do not bounce or hyperextend joints. Safety: Never force end range; joints should feel stable.'}

In [23]:
#We want the LLM to put the document content in the context of the answer
prompt_template = """
    You're a professional fitness assistant. Answer the QUESTION based only on the CONTEXT provided from the exercise & fitness database.  
    
    - Use only the facts from the CONTEXT when answering the QUESTION.  
    - If the CONTEXT does not contain the answer, respond with: NONE.  
    - Keep your answer clear, concise, and detail with instruction for fitness use.  
    
    QUESTION: {question}  
    
    CONTEXT:  
    {context}  
""".strip()

entry_template = """
exercise_name: {exercise_name}',
type_of_activity: {type_of_activity},
type_of_equipment: {type_of_equipment},
body_part: {body_part},
type: {type},
muscle_groups_activated: {muscle_groups_activated},
instruction: {instruction}
""".strip()

def build_prompt(query, search_results):    
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [24]:
def llm(prompt):
    response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
    )

    return response.text

In [25]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [26]:
query = 'I want some core exercises that also help my back'

answer = rag(query)
print(answer)

Here are some core exercises that also help your back:

**1. Wide-Grip Face Pull Twist**
*   **Equipment:** Kettlebells
*   **Muscle Groups Activated:** Lower back, abs, obliques
*   **Instructions:** Place kettlebell close to midline; hinge to grip; pack shoulders. Exhale gently to brace; avoid flaring ribs or arching low back. Pull elbows back toward hips, squeeze shoulder blades, then return slowly without shrugging. Keep neck long, ribs down, and shoulders down. Avoid flaring elbows or arching the back. Avoid collapsing posture or losing core brace. Keep bell close to centerline to protect lower back.
*   **Dose:** 3–4 sets × 6–10 reps. Rest 45–90s between sets.

**2. Feet-Elevated Deadlift Combo**
*   **Equipment:** Dip belt
*   **Muscle Groups Activated:** Abs, obliques, lower back
*   **Instructions:** Prepare equipment: dip belt. Ensure stable setup and safe load. Exhale gently to brace; avoid flaring ribs or arching low back. Hinge at hips with a flat back, push hips back, the

## Retrieval Evalutation

In [30]:
df_question = pd.read_csv('../data/ground-truth-retrieval.csv')

In [31]:
df_question.head()

,id,question
0,0,What equipment do I need to perform the Push-U...
1,0,How long should I hold each side during the Pu...
2,0,What should I focus on to ensure proper form w...
3,0,What common mistakes should I avoid when doing...
4,0,How should I breathe during the Push-Up Hold t...


In [32]:
ground_truth = df_question.to_dict(orient='records')

In [33]:
ground_truth[0]

{'id': 0,
 'question': 'What equipment do I need to perform the Push-Up Hold exercise?'}

In [34]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [35]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Setup: Prepare equipment: dip belt. Ensure stable setup and safe load. Brace core, keep neutral spine, drive through whole foot. Primary focus: glutes, quads, hamstrings. Movement: Gently enter the stretch until a mild pull is felt; hold without bouncing and keep breathing. Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Slow nasal breaths; exhale to deepen slightly, no pain. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid bouncing; stay within mild discomfort. Do not bounce or hyperextend joints. Safety: Never force end range; joints should feel stable.'}

In [36]:
def minsearch_search(query):
    # boost = {'exercise_name':3, 'muscle_groups_activated':1}
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [37]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['id']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [38]:
from tqdm.auto import tqdm

In [29]:
evaluate(ground_truth, lambda q: minsearch_search(q['question']))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:13<00:00, 187.52it/s]


{'hit_rate': 0.6324, 'mrr': 0.5803822222222222}

## Finding the best parameters

In [39]:
df_validation = df_question[:250]
df_test = df_question[250:]

In [40]:
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from hyperopt.pyll import scope

In [41]:
import random

def simple_optimize(param_ranges, objective_function, n_iterations=10):
    best_params = None
    best_score = float('-inf')  # Assuming we're minimizing. Use float('-inf') if maximizing.

    for _ in range(n_iterations):
        # Generate random parameters
        current_params = {}
        for param, (min_val, max_val) in param_ranges.items():
            if isinstance(min_val, int) and isinstance(max_val, int):
                current_params[param] = random.randint(min_val, max_val)
            else:
                current_params[param] = random.uniform(min_val, max_val)
        
        # Evaluate the objective function
        current_score = objective_function(current_params)
        
        # Update best if current is better
        if current_score > best_score:  # Change to > if maximizing
            best_score = current_score
            best_params = current_params
    
    return best_params, best_score

In [42]:
gt_val = df_validation.to_dict(orient='records')

In [43]:
def minsearch_search(query, boost=None):
    if boost is None:
        boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [44]:
param_ranges = {
    'exercise_name': (0.0, 3.0),
    'type_of_activity': (0.0, 3.0),
    'type_of_equipment': (0.0, 3.0),
    'body_part': (0.0, 3.0),
    'type': (0.0, 3.0),
    'muscle_groups_activated': (0.0, 3.0),
    'instruction': (0.0, 3.0),
}

def objective(boost_params):
    def search_function(q):
        return minsearch_search(q['question'], boost_params)

    results = evaluate(gt_val, search_function)
    return results['mrr']

In [40]:
simple_optimize(param_ranges, objective, n_iterations=20)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 250/250 [00:01<00:00, 168.38it/s]


({'exercise_name': 2.4883345828618713,
  'type_of_activity': 1.3629230346257846,
  'type_of_equipment': 2.227694809207545,
  'body_part': 0.5312951324427708,
  'type': 2.530050595897771,
  'muscle_groups_activated': 0.05965456620525866,
  'instruction': 0.5190037713702332},
 0.6020777777777778)

The MRR is 2% better than the basic non-boosting method: 'mrr': 0.5803822222222222

In [46]:
def minsearch_improved(query, boost=None):
    if boost is None:
        boost = {
            'exercise_name': 2.5,
            'type_of_activity': 1.4,
            'type_of_equipment': 2.2,
            'body_part': 0.5,
            'type': 2.5,
            'muscle_groups_activated': 0.06,
            'instruction': 0.5
        }

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

In [ ]:
evaluate(ground_truth, lambda q: minsearch_improved(q['question']))

## RAG evaluation

In [47]:
prompt2_template = """
You are an expert evaluator for a (RAG) system.
Your task is to analyze the relevance of the generated answer to the given question.
Based on the relevance of the generated answer, you will classify it
as "NON_RELEVANT", "PARTLY_RELEVANT", or "RELEVANT".

Here is the data for evaluation:

Question: {question}
Generated Answer: {answer_llm}

Please analyze the content and context of the generated answer in relation to the question
and provide your evaluation in parsable JSON without using code blocks:

{{
  "Relevance": "NON_RELEVANT" | "PARTLY_RELEVANT" | "RELEVANT",
  "Explanation": "[Provide a brief explanation for your evaluation]"
}}
""".strip()

In [48]:
ground_truth[0]

{'id': 0,
 'question': 'What equipment do I need to perform the Push-Up Hold exercise?'}

In [41]:
record = ground_truth[0]
question = record['question']
answer_llm = rag(question)

In [49]:
minsearch_improved('What equipment do I need to perform the Push-Up Hold exercise?')

[{'id': 139,
  'exercise_name': 'Step-Up Hold',
  'type_of_activity': 'warm-up',
  'type_of_equipment': 'kettlebells',
  'body_part': 'lower',
  'type': 'hold',
  'muscle_groups_activated': 'quads, calves, glutes',
  'instruction': 'Setup: Place kettlebell close to midline; hinge to grip; pack shoulders. Brace core, keep neutral spine, drive through whole foot. Primary focus: quads, calves, glutes. Movement: Hold/maintain to a controlled end range, then return under control (Find a stable, aligned position and sustain tension without shaking.). Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Breathe light and steady; do not brace with a breath hold. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid holding breath; keep steady tension. Avoid collapsing posture or losing core brace. Safety: Keep bell close to centerline to protect lower back.'},
 {'id': 204,
  'exercise_name': 'Explosive Farmer Carry'

In [ ]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [37]:
question

'What equipment do I need to perform the Push-Up Hold exercise?'

In [40]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Up Hold',
 'type_of_activity': 'mobility',
 'type_of_equipment': 'dip belt',
 'body_part': 'lower',
 'type': 'stretch',
 'muscle_groups_activated': 'glutes, quads, hamstrings',
 'instruction': 'Setup: Prepare equipment: dip belt. Ensure stable setup and safe load. Brace core, keep neutral spine, drive through whole foot. Primary focus: glutes, quads, hamstrings. Movement: Gently enter the stretch until a mild pull is felt; hold without bouncing and keep breathing. Tempo: 2‑2‑2‑0. Dose: 1–2 sets × 30–60 sec each side. Rest 45–90s between sets. Breathing: Slow nasal breaths; exhale to deepen slightly, no pain. Coaching cues: keep neck long, ribs down, and knees tracking over toes. Common mistakes: avoid bouncing; stay within mild discomfort. Do not bounce or hyperextend joints. Safety: Never force end range; joints should feel stable.'}

In [ ]:
prompt = prompt2_template.format(**record)
print(prompt)